In [77]:
%pip install clean-text


In [78]:
%pip install nltk

%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [79]:
#################################### PACKAGES ######################################
import pandas as pd
from cleantext import clean
import re

import nltk
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import *


R = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv") 


####################################### CLEANING ######################################
Copy_clean = R.copy()

def cleaningTime(x):
    clean_text = x.replace('|', '')
    clean_text = clean_text.replace('$', '')
    regex = r'[xX]+'
    clean_text = re.sub(regex, '', clean_text)
    # Replace date with DATE
    regex = r'\d{1,2}\/\d{1,2}\/\d{2,4}'
    clean_text = re.sub(regex, 'DATE', clean_text)
    clean_text = clean(clean_text,
        no_punct=True,
        lower=True,                    # lowercase text
        no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
        no_urls=True,                  # replace all URLs with a special token
        no_emails=True,                # replace all email addresses with a special token
        no_numbers=True,
        replace_with_url="url",
        replace_with_email="email",
        replace_with_number="number",
        )
    return clean_text

#R['content'].apply(cleaning)
Copy_clean['content'] = R['content'].apply(cleaningTime)
#print(Copy_clean['content'])

# (r'C:\Users\karen\nein.csv', index= None, header=True)
df = pd.DataFrame(Copy_clean)
df.to_csv('copy_clean.csv', index= None, header=True)


####################################### TOKENIZE #########################################
Copy_tok = Copy_clean.copy()

def Tokenize(x):     
    tokens = nltk.word_tokenize(x)
    return tokens

Copy_tok['content'] = Copy_tok['content'].apply(Tokenize)
print(Copy_tok['content'])

#Copy_stop = []
#for i in Copy_tok['content']:
    #tokens = nltk.word_tokenize(i)
    #Copy_stop.append(tokens)
#print(Copy_stop)

######################################## REMOVING STOPWORDS ################################
Copy_stop = Copy_tok.copy()
stop_words = set(stopwords.words('english'))
print("Stop words length: ", len(stop_words))
print("Stop words dictionary: ", (stop_words))
# removing
def Remove_stopwords(x): # x = Copy_tok['content']
    filtered_sentence = []
    for w in x:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

Copy_stop['content'] = Copy_stop['content'].apply(Remove_stopwords)
#print(Copy_stop['content'])


######################################### STEMMING ##########################################
Copy_stemming = Copy_stop.copy()
stemmer = SnowballStemmer("english")

def Stemming_Words(x):
    stem_words = []
    for w in x:
        word = stemmer.stem(w)
        stem_words.append(word)
    return stem_words
Copy_stemming['content'] = Copy_stemming['content'].apply(Stemming_Words)
#print(Copy_stemming['content'])

##############################################################################################
# number of tokens before removing stopwords
def unique_words (file):
    UN = {}
    for row in file:
        for word in row:
            if word not in UN:
                UN[word] = 1
            else: 
                UN[word] += 1
    UN = dict(sorted(UN.items(), key = lambda kv: kv[1], reverse=True)) # Sorting the unique words after number of occurrences, from highest to lowest
    return len(UN)


# Difference between number of tokens before and after removing stopwords
print("Difference between number of tokens before and after removing stopwords: ", (unique_words(Copy_tok['content'])-unique_words(Copy_stop['content'])))

# number of tokens after tokenizing
print("Number of words after tokenizing: ", unique_words(Copy_tok['content']))

# number of tokens after removing stopwords
print("Number of words after removing stopwords: ", unique_words(Copy_stop['content']))

# reduction rate after removing stopwords
print("Reduction rate after removing stopwords: ", 100 - unique_words(Copy_stop['content'])/unique_words(Copy_tok['content']) * 100)

# number of tokens after stemming
print("Number of words after stemming: ", unique_words(Copy_stemming['content']))

# reduction rate after stemming
print("Reduction rate after stemming: ", 100 - unique_words(Copy_stemming['content'])/unique_words(Copy_stop['content']) * 100)



Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0      [sometimes, the, power, of, christmas, will, m...
1      [awakening, of, number, strands, of, dna, reco...
2      [never, hike, alone, a, friday, the, 13th, fan...
3      [when, a, rare, shark, was, caught, scientists...
4      [donald, trump, has, the, unnerving, ability, ...
                             ...                        
245    [prison, for, rahm, gods, work, and, many, oth...
246    [number, useful, items, for, your, tiny, home,...
247    [former, cia, director, michael, hayden, said,...
248    [antonio, sabato, jr, says, hollywoods, libera...
249    [former, us, president, bill, clinton, on, mon...
Name: content, Length: 250, dtype: object
Stop words length:  179
Stop words dictionary:  {'into', 'no', "won't", 'him', 'an', 'under', 'through', 'll', "should've", 'other', 'above', 'will', "mustn't", 'doing', 'during', "couldn't", 'had', 'i', 'hers', 'why', 'just', 'but', 'is', "it's", "that'll", 'on', 'she', 'was', 'they', 'herself', 'has', 'himself', 'very', 's', 'm

Notes:

* The URLs are not correctly formatted: Each URL includes the first word in the content!!!
* Not sure whether the counts below are done in the right order
* Regarding dates: We need to choose which column to use: (1) scraped_at, (2) inserted_at, (3) updated_at
* Insert more "replace" statements in CleaningTime (i.e. DATE)

In [80]:

# # COUNTING
# count = 0 
# wordlist = ['url', 'number']
# for i in Copy_stemming['content']:
#     for x in i:
#         if x in wordlist:
#             count += 1
#         else:
#             count = count
# #count


# Count URLs
count_url = 0
for i in Copy_stemming['content']:
    for x in i:
        if x == 'url':
            count_url += 1
        else:
            count_url = count_url
print("Number of URLs: ", count_url)

# Count dates
count_date = 0
for i in Copy_stemming['content']:
    for x in i:
        if x == 'date':
            count_date += 1
        else:
            count_date = count_date
print("Number of dates: ", count_date)

# Count numbers (numeric values)
count_number = 0
for i in Copy_stemming['content']:
    for x in i:
        if x == 'number':
            count_number += 1
        else:
            count_number = count_number
print("Number of numeric values: ", count_number)

# Count punctuation: check if punctuations are removed
count_punct = 0
for i in Copy_stemming['content']:
    for x in i:
        if x == 'punct':
            count_punct += 1
        else:
            count_punct = count_punct
print("Number of punctuations (check): ", count_punct)

# # Print the date of the oldest article by using 'inserted_at' column
# print("The date of the oldest article: ", min(R['inserted_at']))





Number of URLs:  242
Number of dates:  27
Number of numeric values:  2019
Number of punctuations (check):  0


In [93]:
import csv
import pandas as pd
#csv = open('news_cleaned_2018_02_13.csv')
#csv_ = open('1mio-raw.csv')
#df = pd.read_csv('news_cleaned_2018_02_13.csv')
#df = pd.read_csv('news_cleaned_2018_02_13.csv',sep=',', header = None, skiprows=1000, chunksize=10)
df_ = pd.read_csv('1mio-raw.csv')

# apply cleaningtime function to the content column on a copy of the dataframe
# make deep copy of the dataframe
df_copy = df_.copy(deep=True)
# apply cleaningtime function to the content column. Handle attrribute error float has no attribute replace
for x in df_copy['content'][:100]:
    try:
        df_copy['content'][:100] = df_copy['content'][:100].apply(cleaningTime)
    except AttributeError:
        pass

print(df_copy.head(100)['content'])

C:\Users\karen\AppData\Local\Temp\ipykernel_38944\1239446524.py:7: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv('1mio-raw.csv')
C:\Users\karen\AppData\Local\Temp\ipykernel_38944\1239446524.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['content'][:100] = df_copy['content'][:100].apply(cleaningTime)


   Unnamed: 0   id                domain        type  \
0           0    2         express.co.uk       rumor   
1           1    6    barenakedislam.com        hate   
2           2    7    barenakedislam.com        hate   
3           3    8    barenakedislam.com        hate   
4           4    9    barenakedislam.com        hate   
..        ...  ...                   ...         ...   
95         95  114  bipartisanreport.com   clickbait   
96         96  115               awm.com  unreliable   
97         97  116               awm.com  unreliable   
98         98  117               awm.com  unreliable   
99         99  118               awm.com  unreliable   

                                                  url  \
0   https://www.express.co.uk/news/science/738402/...   
1   http://barenakedislam.com/category/donald-trum...   
2   http://barenakedislam.com/category/donald-trum...   
3   http://barenakedislam.com/2017/12/24/more-winn...   
4   http://barenakedislam.com/2017/12/25/o

In [96]:
print(df_copy.head(102)['content'])

0      life is an illusion at least on a quantum leve...
1      unfortunately he hasnt yet attacked her for is...
2      the los angeles police department has been den...
3      the white house has decided to quietly withdra...
4      the time has come to cut off the tongues of th...
                             ...                        
97     martina big a numberyearold white german model...
98     sushi is a trendy dish it comes from japan and...
99     a numberyearold british vlogger has made a nam...
100    (Facebook/Support Kountze Kids Faith)\n\nA Tex...
101    Concentration Camps for African Migrants Block...
Name: content, Length: 102, dtype: object


In [82]:
#df_.drop('id', inplace=True, axis=1)
#df_.drop('inserted_at', inplace=True, axis=1)
#df_.drop('scraped_at', inplace=True, axis=1)
#df_.drop('updated_at', inplace=True, axis=1)
#df_.drop('summary', inplace=True, axis=1)
#df_.drop('tags', inplace=True, axis=1)
#df_.drop('source', inplace=True, axis=1)
df_

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN
1,1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
2,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
3,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN
4,4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",NaN,[''],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,9929,1170116,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976IBA...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976IBADAN00030,NaN,NaN,[''],NaN,View Tags,NaN,NaN
999996,9930,1170117,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976BOM...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976BOMBAY00030,NaN,NaN,[''],NaN,View Tags,NaN,NaN
999997,9931,1170118,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976STA...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976STATE087319,NaN,NaN,[''],NaN,View Tags,NaN,NaN
999998,9932,1170119,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974BAN...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1974BANGKO15999,NaN,NaN,[''],NaN,View Tags,NaN,NaN


In [83]:
# Make a new column 'label' and assign 1 to all rows with 'type' = 'fake' or 'conspiracy' or 'bias' or 'junksci' or 'hate' or 'unreliable' or 'rumor' and 0 to all other rows
df_['label'] = df_['type'].apply(lambda x: 1 if x == 'fake' or x == 'conspiracy' or x == 'bias' or x == 'junksci' or x == 'hate' or x == 'unreliable' or x == 'rumor' else 0)
df_

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,label
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN,1
1,1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN,1
2,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN,1
3,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN,1
4,4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",NaN,[''],NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,9929,1170116,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976IBA...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976IBADAN00030,NaN,NaN,[''],NaN,View Tags,NaN,NaN,1
999996,9930,1170117,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976BOM...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976BOMBAY00030,NaN,NaN,[''],NaN,View Tags,NaN,NaN,1
999997,9931,1170118,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976STA...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976STATE087319,NaN,NaN,[''],NaN,View Tags,NaN,NaN,1
999998,9932,1170119,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974BAN...,Tor\n\nTor is an encrypted anonymising network...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1974BANGKO15999,NaN,NaN,[''],NaN,View Tags,NaN,NaN,1


In [84]:
df_.tail(20)
count = 0
for i in df_['label']:
    if i == 1:
        count += 1

count

579268

In [89]:
from cleantext import clean
# cleaning the train set
df_['content'] = df_['content'].apply(cleaningTime)

# cleaning the train set with error handling (if is in instance of string)



AttributeError: 'float' object has no attribute 'replace'

In [85]:
# splitting the dataset into train, validation and test set
train_set = df_.sample(frac=0.8, random_state=0)
new_df = df_.drop(train_set.index)
validation_set = new_df.sample(frac=0.5, random_state=0)
test_set = new_df.drop(validation_set.index)

print(train_set.shape)
print(validation_set.shape)
print(test_set.shape)


(800000, 18)
(100000, 18)
(100000, 18)


AttributeError: 'float' object has no attribute 'replace'

In [ ]:
# baseline model using count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(train_set['content'])
# summarize


ValueError: np.nan is an invalid document, expected byte or unicode string.